In [13]:
import pandas as pd
import numpy as np

# Create an example DataFrame
output = pd.read_csv('mapje/meeting_1/7_mediapipe.csv')
noOfFrames = len(output.Frame.value_counts())
data = {'Frame': list(range(noOfFrames)), 'Timestamp': [None] * noOfFrames}
df = pd.DataFrame(data)

df.loc[df['Frame'] == 0, 'Timestamp'] = '14:04:50:0000'
df.loc[df['Frame'] == 33225, 'Timestamp'] = '14:26:59:0000'



# Convert the known timestamps to pd.Timestamp objects
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%H:%M:%S:%f', errors='coerce')

# Calculate the time difference and the number of frames between the known timestamps
time_diff = df.loc[df['Frame'] == 33225, 'Timestamp'].iloc[0] - df.loc[df['Frame'] == 0, 'Timestamp'].iloc[0]
frame_diff = 33225

# Calculate the average time per frame
avg_time_per_frame = time_diff / frame_diff

# Fill in the timestamps for the rest of the frames
for frame_num in range(1, noOfFrames):
    prev_timestamp = df.loc[df['Frame'] == frame_num - 1, 'Timestamp'].iloc[0]
    curr_timestamp = prev_timestamp + avg_time_per_frame
    df.loc[df['Frame'] == frame_num, 'Timestamp'] = curr_timestamp

# Convert the timestamps back to their original format

df['Timestamp'] = df['Timestamp'].apply(lambda x: x.strftime('%H:%M:%S:%f')[:-3] if not pd.isna(x) else np.nan)



In [24]:
df2[-10:]

,Frame,Timestamp
33232,33232,14:26:59:280
33233,33233,14:26:59:320
33234,33234,14:26:59:360
33235,33235,14:26:59:400
33236,33236,14:26:59:440
33237,33237,14:26:59:480
33238,33238,14:26:59:520
33239,33239,14:26:59:560
33240,33240,14:26:59:600
33241,33241,14:26:59:640


In [17]:
df2 = timeInterpolation(0, 33225, '14:04:50:0000', '14:26:59:0000', 1)

In [16]:
import pandas as pd
import numpy as np

def timeInterpolation(firstKnownFrame, lastKnownFrame, timestamp1, timestamp2, meetingNumber):
# Create an example DataFrame
    output = pd.read_csv(f'mapje/meeting_{meetingNumber}/7_mediapipe.csv')
    noOfFrames = len(output.Frame.value_counts())
    data = {'Frame': list(range(noOfFrames)), 'Timestamp': [None] * noOfFrames}
    df = pd.DataFrame(data)

    # Set the known timestamps for frames 2000 and 28000
    df.loc[df['Frame'] == firstKnownFrame, 'Timestamp'] = timestamp1
    df.loc[df['Frame'] == lastKnownFrame, 'Timestamp'] = timestamp2


    # Convert the known timestamps to pd.Timestamp objects
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%H:%M:%S:%f', errors='coerce')

    # Calculate the time difference and the number of frames between the known timestamps
    time_diff = df.loc[df['Frame'] == lastKnownFrame, 'Timestamp'].iloc[0] - df.loc[df['Frame'] == firstKnownFrame, 'Timestamp'].iloc[0]
    frame_diff = lastKnownFrame - firstKnownFrame

    # Calculate the average time per frame
    avg_time_per_frame = time_diff / frame_diff

    # Fill in the timestamps for the rest of the frames    
    for frame_num in range(0, noOfFrames):  # Start from 0 and end at noOfFrames - 1
        if frame_num == firstKnownFrame:
            continue  # Skip the frame with the known timestamp
        if frame_num > 0:
            # Update the average time per frame if the current frame is after the last known frame
            if frame_num == lastKnownFrame + 1:
                time_diff = df.loc[df['Frame'] == lastKnownFrame, 'Timestamp'].iloc[0] - df.loc[df['Frame'] == firstKnownFrame, 'Timestamp'].iloc[0]
                frame_diff = noOfFrames - 1 - firstKnownFrame
                avg_time_per_frame = time_diff / frame_diff

            prev_timestamp = df.loc[df['Frame'] == frame_num - 1, 'Timestamp'].iloc[0]
            curr_timestamp = prev_timestamp + avg_time_per_frame
            df.loc[df['Frame'] == frame_num, 'Timestamp'] = curr_timestamp

    # Convert the timestamps back to their original format
    df['Timestamp'] = df['Timestamp'].apply(lambda x: x.strftime('%H:%M:%S:%f')[:-3] if not pd.isna(x) else np.nan)
    return df



In [10]:
def createLabels(originalDF,meetingNumber=0):
    labels_df = pd.read_excel(f"Meetings/meeting_{meetingNumber}/excelChat.xlsx")
    for person in labels_df.columns:
        if person == 'timestamp':
            continue
        newDF = originalDF.copy(deep=True)
        newDF['Timestamp'] = pd.to_datetime(newDF['Timestamp'], format='%H:%M:%S:%f')
        newDF['Label'] = 'x'
        for index, row in labels_df.iterrows():
            label_timestamp = pd.to_datetime(row['timestamp'], format='%H:%M:%S.%f')
            label_value = row[person]

            # Find the nearest frame by timestamp, rounding down
            nearest_frame = newDF[newDF['Timestamp'] <= label_timestamp].iloc[-1]['Frame']

            # Calculate the start and end frame numbers
            start_frame = max(0, nearest_frame - 500)
            end_frame = nearest_frame + 1

            # Update the 'Label' column for the specified range
            newDF.loc[start_frame:end_frame, 'Label'] = label_value
        newDF['Timestamp'] = newDF['Timestamp'].apply(lambda x: x.strftime('%H:%M:%S:%f')[:-3] if not pd.isna(x) else np.nan)

        # Print the updated dataframe
        print(person)
        newDF.to_csv(f'mapje/meeting_1/{person}.csv', index=False)        
        

In [ ]:
createLabels(df,1)

In [44]:
# labels_df = pd.read_excel("Meetings/meeting_1/excelChat.xlsx")

# # Convert the Timestamp column in your dataframe to datetime
# df_paul['Timestamp'] = pd.to_datetime(df_paul['Timestamp'], format='%H:%M:%S:%f')

# # Create a new column 'Label' and fill it with 'x'
# df_paul['Label'] = 'x'

# # Iterate through each row in the labels_df DataFrame
# for index, row in labels_df.iterrows():
#     label_timestamp = pd.to_datetime(row['timestamp'], format='%H:%M:%S.%f')
#     label_value = row['Paul']

#     # Find the nearest frame by timestamp, rounding down
#     nearest_frame = df_paul[df_paul['Timestamp'] <= label_timestamp].iloc[-1]['Frame']

#     # Calculate the start and end frame numbers
#     start_frame = max(0, nearest_frame - 500)
#     end_frame = nearest_frame + 1

#     # Update the 'Label' column for the specified range
#     df_paul.loc[start_frame:end_frame, 'Label'] = label_value
# df_paul['Timestamp'] = df_paul['Timestamp'].apply(lambda x: x.strftime('%H:%M:%S:%f')[:-3] if not pd.isna(x) else np.nan)

# # Print the updated dataframe
# df_paul.to_csv('mapje/meeting_1/paul.csv', index=False)